<a href="https://colab.research.google.com/github/kkidia/Crop-Type-classfication_Senengal_DL/blob/main/S2_Data_Extraction_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reading corner
SEN4STAT/ESA: https://www.esa-sen4stat.org/user-stories/senegal-prototype/

EOSTAT/FAO: https://data.apps.fao.org/catalog/dataset/5c377b2b-3c2e-4b70-afd7-0c80900b68bb/resource/50bc9ff5-95d2-40cd-af12-6aee2cfcc4ae

RNN: https://www.sciencedirect.com/science/article/pii/S0034425721003230#bb0310 and its GitHub : https://github.com/0zgur0/multi-stage-convSTAR-network/tree/master

https://www.sciencedirect.com/science/article/pii/S0034425724001214

https://www.sciencedirect.com/science/article/abs/pii/S2352938522001203

crop type mapping in Ghana and South Sudan: GitHubCode: https://github.com/roserustowicz/crop-type-mapping


In [1]:
#Authotication
import ee
# @title Authenticate to the Earth Engine servers
ee.Authenticate()
# Initialize the Earth Engine object with Google Cloud project ID
project_id = 'ee-kkidia3' # change here
ee.Initialize(project=project_id)

In [ ]:
pip install rasterio

In [2]:
# @title Lib imports:
#import ee
#print('Using EE version ', ee.__version__)
import folium
from folium.plugins import DualMap
from branca.element import Template, MacroElement
#print('Using Folium version ', folium.__version__)
from os import MFD_HUGE_1MB
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, Iterable, List, Tuple
#from google.colab import auth
import datetime as dt
import time
import numpy as np

import geopandas as gpd
from shapely.geometry import shape, Polygon, MultiPolygon
from shapely import wkt

from google.cloud import storage
import tensorflow as tf
import json

In [5]:
#@title Clean data 18, 19, 20, and 2023
def load_cleandata(asset_id):
    return ee.FeatureCollection(asset_id)
clean2018 = load_cleandata('projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2018')
clean2019 = load_cleandata('projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2019')
clean2020 = load_cleandata('projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2020')
clean2023 = load_cleandata('projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2023')

asset_clean = {
    'clean2018': 'projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2018',  # clean Data 2018
    'clean2019': 'projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2019',  # clean Data 2019
    'clean2020': 'projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2020',  # clean Data 2020
    'clean2023': 'projects/ee-kkidia3/assets/clean_data_2018-2023_no-bands/clean_raw_data_2023',  #clean Data 2023
}
clean_data = clean2018.merge(clean2019).merge(clean2020).merge(clean2023)

In [ ]:
print(clean_data.size().getInfo())
print(clean2019.size().getInfo())

17836
1433


In [ ]:
# @title Read polygons properties

# Define a function to fetch features in batches
def get_feature_batch(collection, batch_size=5000):
    # Initialize an empty list to store all features
    all_features = []

    # Get the total number of features in the collection
    total_count = collection.size().getInfo()

    # Loop through the collection in batches
    for i in range(0, total_count, batch_size):
        # Fetch the current batch of features
        batch = collection.toList(batch_size, i).map(lambda f: ee.Feature(f).toDictionary()).getInfo()

        # Add the batch to the list of all features
        all_features.extend(batch)

    return all_features

# Use the function to get all features from the collection
feature_properties = get_feature_batch(ee.FeatureCollection(clean_data))

# Convert to DataFrame
df = pd.DataFrame(feature_properties)

# Show the first few rows
df.head()


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

In [ ]:
# Extract Labels
import pandas as pd

# Function to fetch features in batches
def get_feature_batch(collection, batch_size=5000):
    # Initialize an empty list to store all features
    all_features = []

    # Get the total number of features in the collection
    total_count = collection.size().getInfo()

    # Loop through the collection in batches
    for i in range(0, total_count, batch_size):
        # Fetch the current batch of features
        batch = collection.toList(batch_size, i).map(lambda f: ee.Feature(f).toDictionary()).getInfo()

        # Add the batch to the list of all features
        all_features.extend(batch)

    return all_features

# Use the function to get all features from the collection
feature_properties = get_feature_batch(ee.FeatureCollection(clean_data))

# Convert to DataFrame
df = pd.DataFrame(feature_properties)

# Group the DataFrame by "Name", and count occurrences, keeping "Class" and "Sub-class"
grouped_df = df.groupby(['Name', 'Class', 'Sub_class']).size().reset_index(name='Count')

# Export the grouped data to a CSV file
grouped_df.to_csv('grouped_polygons.csv', index=False)

# To display the resulting CSV file using Google Colab sheets
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=grouped_df)

# Show the first few rows
grouped_df.head()


In [44]:
#@title Extract Sentinel temporal data

# Load polygon asset.
polygons = ee.FeatureCollection('projects/ee-kkidia3/assets/crop2023/okra')  # simple dataset for trial and error

# Define the date range for August, September, and October of 2019.
start_date = '2019-08-01'
end_date = '2019-10-31'

# Load the Sentinel-2 Harmonized ImageCollection.
s2_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterDate(start_date, end_date) \
    .filterBounds(polygons) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .select(['B2', 'B3', 'B4', 'B8', 'QA60'])  # Blue, Green, Red, NIR bands at 10m resolution + QA60 for cloud masking.

# Function to mask clouds and cirrus using the QA60 band
def mask_s2_clouds(image):
    qa = image.select('QA60')  # Select the QA60 band

    # Bits 10 and 11 are for clouds and cirrus respectively
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Create a mask for clear conditions
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    # Return the masked image
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])

# Function to normalize the pixel values of each band (min-max scaling).
def normalize_bands(image):
    band_min_max = {
        'B2': (0, 10000),  # Min and max values for Blue band
        'B3': (0, 10000),  # Min and max values for Green band
        'B4': (0, 10000),  # Min and max values for Red band
        'B8': (0, 10000),  # Min and max values for NIR band
    }

    # Normalize each band (standard min-max scaling).
    normalized = ee.Image([
        image.select(band).subtract(band_min_max[band][0]).divide(band_min_max[band][1] - band_min_max[band][0])
        for band in ['B2', 'B3', 'B4', 'B8']
    ]).rename(['B2', 'B3', 'B4', 'B8'])

    return normalized

# Apply cloud masking and normalization to the image collection
processed_collection = s2_collection.map(mask_s2_clouds).map(normalize_bands)



In [45]:
#@title count number of imegeries per polygon

# Function to check which polygons have imagery and count the number of images
def check_imagery_count(feature):
    # Get the patch geometry based on the feature (polygon)
    centroid = feature.geometry().centroid()
    buffer_distance = 320  # Buffer distance to match 640m x 640m patch
    patch_geometry = centroid.buffer(buffer_distance).bounds()

    # Filter images that intersect the patch geometry
    images = processed_collection.filterBounds(patch_geometry)

    # Get the number of images for this polygon
    image_count = images.size()

    # Return the feature with the image count as a new property
    # Removed the client-side operations (print and getInfo) from within the mapped function
    return feature.set({'image_count': image_count})

# Map the function over all polygons to check imagery availability
polygons_with_imagery = polygons.map(check_imagery_count)

# Print out the total number of images for each polygon
polygon_info = polygons_with_imagery.aggregate_array('image_count').getInfo()
total_polygons = len(polygon_info)
polygons_with_images = sum(1 for count in polygon_info if count > 0)

print(f"Total polygons: {total_polygons}")
print(f"Polygons with imagery: {polygons_with_images}")
print(f"Polygons without imagery: {total_polygons - polygons_with_images}")

# Iterate over the features in the collection and print the information
for i in range(total_polygons):
  if polygon_info[i] > 0:
    print(f"Polygon ID: {i} has {polygon_info[i]} images.")
  else:
    print(f"Polygon ID: {i} has no images.")

Total polygons: 9
Polygons with imagery: 9
Polygons without imagery: 0
Polygon ID: 0 has 7 images.
Polygon ID: 1 has 7 images.
Polygon ID: 2 has 3 images.
Polygon ID: 3 has 4 images.
Polygon ID: 4 has 5 images.
Polygon ID: 5 has 5 images.
Polygon ID: 6 has 5 images.
Polygon ID: 7 has 5 images.
Polygon ID: 8 has 7 images.


Export Sentinel imegeries to Pytorch data environment

here is the steps:
1. Include Polygon Labels: Add polygon labels as metadata in the exported TFRecord.
2. Spatial Metadata: Include Coordinate Reference Systems (CRS) and Affine transformations in the GEOTIF.
3. Normalization of Bands: Normalize the pixel values of the spectral bands.
4. Batching and Prefetching: Adjust the export logic to avoid overlaps and improve performance by batching and prefetching.

In [ ]:
#@title Export as GeoTIFF
# Function to create a fixed 64x64 pixel patch with consistent size and projection
def create_fixed_size_patch(feature):
    # Get the centroid of the polygon
    centroid = feature.geometry().centroid()

    # Define the buffer distance as 320 meters to create a 640m x 640m patch
    buffer_distance = 320  # Half of the patch size in meters
    patch_geometry = centroid.buffer(buffer_distance).bounds()

    # Define a consistent UTM projection, adjust UTM zone based on the region (e.g., EPSG:32630 for West Africa)
    utm_projection = ee.Projection('EPSG:32630').atScale(10)  # 10-meter resolution

    # Filter images that intersect the patch geometry
    images = s2_collection.filterBounds(patch_geometry)

    # Apply cloud masking and normalization
    images = images.map(mask_s2_clouds).map(normalize_bands)

    def clip_and_reproject_image(image):
        # Clip the image to the fixed-size geometry (640m x 640m)
        clipped_image = image.clip(patch_geometry)

        # Reproject to the defined UTM projection with a 10m resolution
        reprojected_image = clipped_image.reproject(utm_projection)

        # Get the 'system:time_start' property
        date = image.get('system:time_start')

        # Format the date if available, otherwise use an empty string for the date part
        formatted_date = ee.Algorithms.If(date, ee.Date(date).format('YYYY-MM-dd'), '')

        # Set metadata to ensure each image has consistent affine transformation and CRS
        reprojected_image = reprojected_image.set({
            'polygon_id': feature.id(),
            'label': feature.get('label'),
            'date': formatted_date,  # Use the exact imagery date if available, else ''
            'crs': utm_projection.crs(),
            'affine': utm_projection.transform(),
            'system_index': image.get('system:index')
        })

        return reprojected_image

    # Apply the clipping and reproject function to each image
    patches = images.map(clip_and_reproject_image)
    return patches

# Map the create_fixed_size_patch function over all polygons and flatten the result into an ImageCollection
all_patches = polygons.map(create_fixed_size_patch).flatten()
all_patches = ee.ImageCollection(all_patches)

# Get the total number of patches
patch_count = all_patches.size().getInfo()
print(f'Total patches to export: {patch_count}')

# Convert the collection to a list for iteration
patches_list = all_patches.toList(patch_count)

# Function to export each patch as GeoTIFF with explicit region size and projection
def export_fixed_patch(index):
    image = ee.Image(patches_list.get(index))
    polygon_id = image.get('polygon_id').getInfo()
    date = image.get('date').getInfo() or ""  # If no date, use an empty string
    system_index = image.get('system_index').getInfo()

    # Create the filename, include the date only if it's not empty
    filename = f's2_patch_{polygon_id}{"_" + date if date else ""}_{system_index}'

    # Define the export task with an explicit region (640m x 640m) and consistent UTM projection
    task = ee.batch.Export.image.toCloudStorage(
        image=image.toFloat(),
        description=filename,
        bucket='okra_foo',  # your bucket name
        scale=10,  # 10-meter resolution
        region=image.geometry().bounds().getInfo()['coordinates'],  # Explicitly set region
        crs='EPSG:32630',  # Consistent UTM projection
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized': True  # Optimized for cloud storage
        }
    )
    task.start()
    print(f'Exporting {filename} as GeoTIFF')

# Loop over the patches and export each one in batches
batch_size = 10  # Adjust batch size as needed
for i in range(0, patch_count, batch_size):
    for j in range(i, min(i + batch_size, patch_count)):
        export_fixed_patch(j)
    print(f'Batch {i // batch_size + 1} exported.')


In [ ]:
pip install rasterio

In [ ]:

import rasterio
from rasterio.io import MemoryFile

# Initialize Google Cloud Storage client
client = storage.Client()

# Define the bucket and file name
bucket_name = 'okra_foo'  # Your GCS bucket
file_name = 'planet_patch_00000000000000000007_20191027T112119_20191027T112939_T28PEV.tif'  # The GeoTIFF file in your GCS bucket

# Access the file in the Google Cloud Storage bucket
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Read the file into memory without downloading
geo_tiff_data = blob.download_as_bytes()

# Use rasterio to read the GeoTIFF from memory
with MemoryFile(geo_tiff_data) as memfile:
    with memfile.open() as dataset:
        # Read the image data
        image_data = dataset.read()  # Reads all bands into a numpy array

        # Get metadata from the GeoTIFF
        metadata = dataset.meta

        # Print image shape and metadata
        print(f"Image shape: {image_data.shape}")  # (bands, height, width)
        print("Metadata:", metadata)

        # Print individual metadata items, e.g., CRS (Coordinate Reference System)
        print(f"CRS: {dataset.crs}")
        print(f"Transform: {dataset.transform}")

        # You can also access other metadata fields
        tags = dataset.tags()
        print("Tags:", tags)


In [ ]:
#@title RGB
import gcsfs
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Initialize Google Cloud Storage file system
gcs = gcsfs.GCSFileSystem()

# Define your bucket and file path in Google Cloud Storage
bucket_name = 'okra_foo'  # Replace with your actual bucket name
file_name = 'planet_patch_00000000000000000007_20191027T112119_20191027T112939_T28PEV.tif'  # Replace with your actual file name
gcs_path = f'gs://{bucket_name}/{file_name}'

# Use gcsfs to open the file as a file-like object
with gcs.open(gcs_path, 'rb') as f:
    # Open the GeoTIFF file using rasterio
    with rasterio.open(f) as dataset:
        # Read the RGB bands (assuming Band 4 = Red, Band 3 = Green, Band 2 = Blue)
        red = dataset.read(4)  # Band 4: Red
        green = dataset.read(3)  # Band 3: Green
        blue = dataset.read(2)  # Band 2: Blue

        rgb_image = np.dstack((red, green, blue))

        # Plot the RGB image using matplotlib
        plt.figure(figsize=(10, 10))
        plt.imshow(rgb_image)
        plt.title('Sentinel-2 RGB Image from Google Cloud Storage')
        plt.axis('off')  # Hide axes
        plt.show()


In [ ]:
#@title False color
import gcsfs
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Initialize Google Cloud Storage file system
gcs = gcsfs.GCSFileSystem()

# Define your bucket and file path in Google Cloud Storage
bucket_name = 'okra_foo'  # Replace with your actual bucket name
file_name = 'planet_patch_00000000000000000007_20191027T112119_20191027T112939_T28PEV.tif'  # Replace with your actual file name
gcs_path = f'gs://{bucket_name}/{file_name}'

# Use gcsfs to open the file as a file-like object
with gcs.open(gcs_path, 'rb') as f:
    # Open the GeoTIFF file using rasterio
    with rasterio.open(f) as dataset:
        # Check the number of available bands
        num_bands = dataset.count
        print(f'This GeoTIFF contains {num_bands} bands.')

        # Print band descriptions (if available)
        for i in range(1, num_bands + 1):
            print(f'Band {i}: {dataset.descriptions[i - 1]}')

        # Adapt based on the number of available bands
        if num_bands >= 4:
            # Read the normalized Red, Green, and NIR bands for false color display
            # Assuming the file has RGB (bands 1, 2, 3) and NIR (band 4)
            red_band = dataset.read(3)  # Read Band 3 (assumed to be Red)
            green_band = dataset.read(2)  # Read Band 2 (assumed to be Green)
            nir_band = dataset.read(4)  # Read Band 4 (assumed to be NIR)

            # Stack bands to create an RGB image for false color (NIR, Red, Green)
            false_color_image = np.stack((nir_band, red_band, green_band), axis=-1)

            # Display the false color image using matplotlib
            plt.figure(figsize=(10, 10))
            plt.imshow(false_color_image)
            plt.title('Sentinel-2 False Color Image (NIR, Red, Green)')
            plt.axis('off')
            plt.show()
        else:
            print(f"Not enough bands for a false-color image. This GeoTIFF has {num_bands} band(s).")


In [ ]:
#@title Compare 2 Patches /RGB
import gcsfs
import rasterio
import matplotlib.pyplot as plt
import numpy as np

# Initialize Google Cloud Storage file system
gcs = gcsfs.GCSFileSystem()

# Define your bucket and file paths in Google Cloud Storage for two images
bucket_name = 'okra_foo'  # Replace with your actual bucket name
file_name_1 = 's2_patch_00000000000000000001_20190811T113329_20190811T113716_T28PCB.tif'  # Replace with the first file name
file_name_2 = 'planet_patch_00000000000000000007_20191027T112119_20191027T112939_T28PEV.tif'  # Replace with the second file name
gcs_path_1 = f'gs://{bucket_name}/{file_name_1}'
gcs_path_2 = f'gs://{bucket_name}/{file_name_2}'

# Function to read RGB bands (Red=4, Green=3, Blue=2) from a GeoTIFF in GCS
def read_rgb_image(gcs_path):
    with gcs.open(gcs_path, 'rb') as f:
        with rasterio.open(f) as dataset:
            # Read the RGB bands
            red = dataset.read(4)  # Band 4: Red
            green = dataset.read(3)  # Band 3: Green
            blue = dataset.read(2)  # Band 2: Blue
            # Stack the bands into a 3D numpy array
            rgb_image = np.dstack((red, green, blue))
            return rgb_image

# Read the two GeoTIFF images
rgb_image_1 = read_rgb_image(gcs_path_1)
rgb_image_2 = read_rgb_image(gcs_path_2)

# Create a figure with two subplots (1 row, 2 columns) to display the images side by side
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

# Display the first image
axes[0].imshow(rgb_image_1)
axes[0].set_title('Sentinel-2 RGB Image 1')
axes[0].axis('off')  # Hide the axes

# Display the second image
axes[1].imshow(rgb_image_2)
axes[1].set_title('Sentinel-2 RGB Image 2')
axes[1].axis('off')  # Hide the axes

# Show the plot with both images side by side
plt.tight_layout()
plt.show()


In [8]:
#@title NICFI
# Load polygon asset.
polygons = ee.FeatureCollection('projects/ee-kkidia3/assets/crop2023/okra')  # simple dataset for trial and error

# Define the date range for August, September, and October of 2019.
start_date = '2019-08-01'
end_date = '2019-10-31'
# Load the NICFI/Planet ImageCollection.
planet_collection = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/africa') \
    .filterDate(start_date, end_date) \
    .filterBounds(polygons) \
    .select(['R', 'G', 'B', 'N'])  # Red, Green, Blue, NIR bands
# Since NICFI/Planet does not provide QA60 cloud mask, we will skip the cloud masking part.
def normalize_bands(image):
    band_min_max = {
        'R': (0, 255),  # Min and max values for Red band
        'G': (0, 255),  # Min and max values for Green band
        'B': (0, 255),  # Min and max values for Blue band
        'N': (0, 255),  # Min and max values for NIR band
    }

    # Normalize each band (standard min-max scaling).
    normalized = ee.Image([
        image.select(band).subtract(band_min_max[band][0]).divide(band_min_max[band][1] - band_min_max[band][0])
        for band in ['R', 'G', 'B', 'N']
    ]).rename(['R', 'G', 'B', 'N'])

    return normalized
# Apply normalization to the NICFI/Planet image collection
processed_collection = planet_collection.map(normalize_bands)
def create_fixed_size_patch(feature):
    # Get the centroid of the polygon
    centroid = feature.geometry().centroid()

    # Define the buffer distance as 320 meters to create a 640m x 640m patch
    buffer_distance = 320  # Half of the patch size in meters
    patch_geometry = centroid.buffer(buffer_distance).bounds()

    # Define a consistent UTM projection, adjust UTM zone based on the region (e.g., EPSG:32630 for West Africa)
    utm_projection = ee.Projection('EPSG:32630').atScale(4)  # 4-meter resolution for PlanetScope

    # Filter images that intersect the patch geometry
    images = planet_collection.filterBounds(patch_geometry)

    # Apply normalization
    images = images.map(normalize_bands)

    def clip_and_reproject_image(image):
        # Clip the image to the fixed-size geometry (640m x 640m)
        clipped_image = image.clip(patch_geometry)

        # Reproject to the defined UTM projection with a 4m resolution
        reprojected_image = clipped_image.reproject(utm_projection)

        # Get the 'system:time_start' property
        date = image.get('system:time_start')

        # Format the date if available, otherwise use an empty string for the date part
        formatted_date = ee.Algorithms.If(date, ee.Date(date).format('YYYY-MM-dd'), '')

        # Set metadata to ensure each image has consistent affine transformation and CRS
        reprojected_image = reprojected_image.set({
            'polygon_id': feature.id(),
            'label': feature.get('label'),
            'date': formatted_date,  # Use the exact imagery date if available, else ''
            'crs': utm_projection.crs(),
            'affine': utm_projection.transform(),
            'system_index': image.get('system:index')
        })

        return reprojected_image

    # Apply the clipping and reproject function to each image
    patches = images.map(clip_and_reproject_image)
    return patches

# Map the create_fixed_size_patch function over all polygons and flatten the result into an ImageCollection
all_patches = polygons.map(create_fixed_size_patch).flatten()
all_patches = ee.ImageCollection(all_patches)

# Export the patches with appropriate parameters for NICFI/Planet imagery
def export_fixed_patch(index):
    image = ee.Image(patches_list.get(index))
    polygon_id = image.get('polygon_id').getInfo()
    date = image.get('date').getInfo() or ""  # If no date, use an empty string
    system_index = image.get('system_index').getInfo()

    filename = f'planet_patch_{polygon_id}{"_" + date if date else ""}_{system_index}'

    task = ee.batch.Export.image.toCloudStorage(
        image=image.toFloat(),
        description=filename,
        bucket='okra_foo',  # your bucket name
        scale=4,  # 4-meter resolution for PlanetScope imagery
        region=image.geometry().bounds().getInfo()['coordinates'],  # Explicitly set region
        crs='EPSG:32630',  # Consistent UTM projection
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized': True  # Optimized for cloud storage
        }
    )
    task.start()
    print(f'Exporting {filename} as GeoTIFF')


In [9]:
# Loop over the patches and export each one in batches
batch_size = 10  # Adjust batch size as needed
for i in range(0, patch_count, batch_size):
    for j in range(i, min(i + batch_size, patch_count)):
        export_fixed_patch(j)
    print(f'Batch {i // batch_size + 1} exported.')